# Ex17



Uma determinada fábrica produz panelas de metal médias e grandes a partir de elementos circulares de diâmetros de 0,25 e 0,40 metros, respectivamente. A primeira operação para obter as panelas é um corte desses elementos circulares sobre chapas de dimensão de 1,40 x 0,50 metros. Os elementos planos circulares são transformados em panelas em uma segunda operação de estamparia. Para o corte existem quatro tipos de matrizes conforme mostra a figura abaixo,

| MATRIZ | Panela média | Panela Grande |
|--------------|------------|-------------|
| **1**     | 8     | 0      |
| **2**   | 4      | 1      |
| **3**     | 2     | 2      |
| **4**     | 0      | 3      |

A fábrica deseja uma produção diária mínima de 500 panelas médias (obtidas do elemento circular de diâmetro 0,25) e 350 grandes (obtidas do elemento circular de diâmetro 0,40). Os custos em reais por chapa pelo uso de cada matriz de corte são respectivamente: 1, 2, 3, 2. 

Elaborar o modelo de Programação Linear que planeje a produção de modo a minimizar o custo com o uso das chapas.



## **Definição das Variáveis de Decisão**

- $x_j$ = Número de chapas cortadas utilizando a matriz $j$, onde $j \in \{1, 2, 3, 4\}$.



## **Função Objetivo**
O objetivo é **minimizar** o custo total com as chapas:
$$\min Z = 1x_1 + 2x_2 + 3x_3 + 2x_4$$


## **Restrições de Produção**
A fábrica precisa produzir pelo menos **500 panelas médias** e **350 panelas grandes** por dia. Isso gera as seguintes restrições:

### **Produção mínima de panelas médias**
$$8x_1 + 4x_2 + 2x_3 + 0x_4 \geq 500$$

### **Produção mínima de panelas grandes**
$$0x_1 + 1x_2 + 2x_3 + 3x_4 \geq 350$$



## **Restrição de Não-Negatividade**
As quantidades de chapas utilizadas devem ser não negativas:
$$x_j \geq 0, \quad \forall j \in \{1, 2, 3, 4\}$$




In [14]:
import pyomo.environ as pyo

In [15]:
# Criação do modelo
model = pyo.ConcreteModel()

In [16]:

# Conjuntos
model.Matrizes = pyo.Set(initialize=["1", "2", "3", "4"])


In [17]:
# Custo por chapa cortada utilizando cada matriz
custo = {"1": 1, "2": 2, "3": 3, "4": 2}
model.custo_chapa = pyo.Param(model.Matrizes, initialize=custo)

# Produção de panelas médias por matriz
prod_medias = {"1": 8, "2": 4, "3": 2, "4": 0}
model.prod_medias = pyo.Param(model.Matrizes, initialize=prod_medias)

# Produção de panelas grandes por matriz
prod_grandes = {"1": 0, "2": 1, "3": 2, "4": 3}
model.prod_grandes = pyo.Param(model.Matrizes, initialize=prod_grandes)

# Demanda mínima de produção diária
model.demanda_medias = pyo.Param(initialize=500)
model.demanda_grandes = pyo.Param(initialize=350)

In [18]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Matrizes, domain=pyo.NonNegativeIntegers)

In [19]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[j] * model.custo_chapa[j] for j in model.Matrizes)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [20]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Produção mínima de panelas médias
def producao_medias_rule(model):
    return sum(model.x[j] * model.prod_medias[j] for j in model.Matrizes) >= model.demanda_medias

model.producao_medias = pyo.Constraint(rule=producao_medias_rule)

# 2. Produção mínima de panelas grandes
def producao_grandes_rule(model):
    return sum(model.x[j] * model.prod_grandes[j] for j in model.Matrizes) >= model.demanda_grandes

model.producao_grandes = pyo.Constraint(rule=producao_grandes_rule)



In [21]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex17.lp", io_options={"symbolic_solver_labels": True})

('ex17.lp', 1822205702496)

In [22]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 2 rows; 4 cols; 6 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 5e+02]
Presolving model
2 rows, 4 cols, 6 nonzeros  0s
2 rows, 4 cols, 6 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00% 

In [ ]:

for j in model.Matrizes:
    valor = pyo.value(model.x[j])
    print(f"Matriz {j}: {valor:.2f} chapas")

Resultados do modelo:
Matriz 1: 63.00 chapas
Matriz 2: 0.00 chapas
Matriz 3: 0.00 chapas
Matriz 4: 117.00 chapas
